In [39]:
import pandas as pd

### Strong Production

In [40]:
recastPD = pd.read_pickle('./cutFlow_comb_strong.pcl')

In [41]:
atlasCutFlow = {}
atlasCutFlow['$m_{\\tilde g} (GeV)$'] = []
atlasCutFlow['$m_{\\tilde \chi_1^0} (GeV)$'] = []
atlasCutFlow['$\\tau(\\tilde \chi_1^0) (ns)$'] = []
atlasCutFlow.update({"Total" : [],
            "Jet selection" : [],
            "$R_{xy},z <$ 300 mm" : [],
            "$R_{DV} > 4$ mm" : [],
            "$d_0 > 2$ mm" : [],
            "$nTracks >= 5$" : [],
            "mDV > 10 GeV" : [],
            "final Acc*Eff" : []
            })

In [42]:
with open('../ATLAS_data/HEPData-ins2628398-v1-csv/acceptance_highpt_strong.csv', 'r') as f:
    blocks = f.read().split('#: Luminosity [fb$^{-1}$],139')[1:]
for b in blocks:
    lines = [l for l in b.split('\n') if l.strip()]
    header_lines = [l for l in lines if l[0] == '#']
    data_lines = [l for l in lines if l[0] != '#' and 'Selections,Acceptance' not in l]
    mGluino = float([l for l in header_lines if '$m(\\tilde{g})$ [GeV]' in l][0].split(',')[1])
    mLLP = float([l for l in header_lines if '$m(\\tilde{\chi}^0_1)$' in l][0].split(',')[1])
    tau =  float([l for l in header_lines if '$\\tau$ [ns]' in l][0].split(',')[1])
    data = [mGluino,mLLP,tau,1.0]+[float(l.split(',')[-1]) for l in data_lines]+[0.0]
    for ik,k in enumerate(atlasCutFlow):
        atlasCutFlow[k].append(data[ik])
    
    

In [43]:
atlasPD = pd.DataFrame.from_dict(atlasCutFlow, orient='index')
atlasPD.loc['final Acc*Eff'] = [27.8/1e2,14.4/1e2,11.5/1e2,9.2/1e2]

In [44]:
combData = recastPD.copy()
for index in recastPD.index.values:
    recastVals = recastPD.loc[index]
    atlasVals = atlasPD.loc[index]
    if index == 'Total' or '(GeV)' in index or '(ns)' in index:
        combData.loc[index] = [float('%1.3f' %x) for x in recastVals]
        continue
    merged = ['%1.3f (%1.3f)' %x for x in zip(recastVals,atlasVals)]
    # print(index)
    # print(merged)
    combData.loc[index] = merged

/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.999 (0.999)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combData.loc[index] = merged
/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.967 (0.966)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combData.loc[index] = merged
/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.985 (0.972)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combData.loc[index] = merged
/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

In [45]:
print(combData.to_markdown())

|                              | 0             | 1             | 2             | 3             |
|:-----------------------------|:--------------|:--------------|:--------------|:--------------|
| $m_{\tilde g} (GeV)$         | 2000.0        | 2000.0        | 2400.0        | 2000.0        |
| $m_{\tilde \chi_1^0} (GeV)$  | 850.0         | 50.0          | 200.0         | 1250.0        |
| $\tau(\tilde \chi_1^0) (ns)$ | 0.01          | 0.1           | 1.0           | 10.0          |
| Total                        | 1.0           | 1.0           | 1.0           | 1.0           |
| Jet selection                | 0.999 (0.999) | 0.967 (0.966) | 0.985 (0.972) | 0.999 (0.961) |
| $R_{xy},z <$ 300 mm          | 0.999 (0.999) | 0.788 (0.787) | 0.442 (0.447) | 0.311 (0.317) |
| $R_{DV} > 4$ mm              | 0.298 (0.296) | 0.772 (0.770) | 0.434 (0.438) | 0.304 (0.309) |
| $d_0 > 2$ mm                 | 0.296 (0.296) | 0.767 (0.756) | 0.433 (0.437) | 0.304 (0.309) |
| $nTracks >= 5$              

### EWK Production

In [46]:
# recastPD = None
# allDFs = []
# for f in glob.glob('./ewk_cutFlow/Events/run_*/*_cutflow.pcl'):
#     allDFs.append(pd.read_pickle(f))
# allDFs = sorted(allDFs, key = lambda df: df.T[['$m_{\\tilde \chi_1^0} (GeV)$','$\\tau(\\tilde \chi_1^0) (ns)$']].values[0].tolist())    

# recastPD = pd.concat(allDFs,axis=1,ignore_index=True)
# recastPD.to_pickle('ewk_cutFlow.pcl')
recastPD = pd.read_pickle('ewk_cutFlow.pcl')

In [47]:
atlasCutFlow = {}
atlasCutFlow['$m_{\\tilde \chi_1^0} (GeV)$'] = []
atlasCutFlow['$\\tau(\\tilde \chi_1^0) (ns)$'] = []
atlasCutFlow.update({"Total" : [],
            "Jet selection" : [],
            "$R_{xy},z <$ 300 mm" : [],
            "$R_{DV} > 4$ mm" : [],
            "$d_0 > 2$ mm" : [],
            "$nTracks >= 5$" : [],
            "mDV > 10 GeV" : [],
            "final Acc*Eff" : []
            })


In [48]:
with open('../ATLAS_data/HEPData-ins2628398-v1-csv/acceptance_trackless_ewk.csv', 'r') as f:
    blocks = f.read().split('#: Luminosity [fb$^{-1}$],139')[1:]
for b in blocks:
    lines = [l for l in b.split('\n') if l.strip()]
    header_lines = [l for l in lines if l[0] == '#']
    data_lines = [l for l in lines if l[0] != '#' and 'Selections,Acceptance' not in l]
    mLLP = float([l for l in header_lines if '$m(\\tilde{\chi}^0_1)$' in l][0].split(',')[1])
    tau =  float([l for l in header_lines if '$\\tau$ [ns]' in l][0].split(',')[1])
    data = [mLLP,tau,1.0]+[float(l.split(',')[-1]) for l in data_lines]+[0.0]
    for ik,k in enumerate(atlasCutFlow):
        atlasCutFlow[k].append(data[ik])
    
    

In [49]:
atlasPD = pd.DataFrame.from_dict(atlasCutFlow, orient='index')
atlasPD.loc['final Acc*Eff'] = [31.1/1e2,14.3/1e2,12.2/1e2,8.3/1e2]

In [50]:
combData = recastPD.copy()
for index in recastPD.index.values:
    recastVals = recastPD.loc[index]
    atlasVals = atlasPD.loc[index]
    if index == 'Total' or '(GeV)' in index or '(ns)' in index:
        combData.loc[index] = [float('%1.3f' %x) for x in recastVals]
        continue
    merged = ['%1.3f (%1.3f)' %x for x in zip(recastVals,atlasVals)]
    # print(index)
    # print(merged)
    combData.loc[index] = merged

/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.502 (0.495)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combData.loc[index] = merged
/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.513 (0.501)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combData.loc[index] = merged
/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.984 (0.968)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combData.loc[index] = merged
/tmp/ipykernel_154820/2776082500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

In [51]:
print(combData.to_markdown())

|                              | 0             | 1             | 2             | 3             |
|:-----------------------------|:--------------|:--------------|:--------------|:--------------|
| $m_{\tilde \chi_1^0} (GeV)$  | 500.0         | 500.0         | 1300.0        | 1300.0        |
| $\tau(\tilde \chi_1^0) (ns)$ | 0.1           | 1.0           | 0.1           | 1.0           |
| Total                        | 1.0           | 1.0           | 1.0           | 1.0           |
| Jet selection                | 0.502 (0.495) | 0.513 (0.501) | 0.984 (0.968) | 0.986 (0.985) |
| $R_{xy},z <$ 300 mm          | 0.502 (0.495) | 0.416 (0.410) | 0.984 (0.968) | 0.924 (0.921) |
| $R_{DV} > 4$ mm              | 0.465 (0.465) | 0.403 (0.398) | 0.865 (0.859) | 0.905 (0.899) |
| $d_0 > 2$ mm                 | 0.464 (0.465) | 0.402 (0.398) | 0.863 (0.859) | 0.905 (0.899) |
| $nTracks >= 5$               | 0.464 (0.465) | 0.402 (0.398) | 0.863 (0.859) | 0.905 (0.899) |
| mDV > 10 GeV                